# Cell 0: 🎯 Mobile VLA + Kosmos 2B 액션 예측 시스템

## 🔍 **핵심 개선사항**
| 이전 (이벤트 분류) | 현재 (액션 예측) |
|------------------|------------------|
| 🔴 이벤트 99.9% → 너무 단순 | ✅ 이미지→액션 매핑 학습 |
| 🔴 Frame-by-frame | ✅ Window/Chunk 방식 |
| 🔴 Classification 위주 | ✅ Regression 위주 |
| 🔴 단순 PIL 변환 | ✅ RoboVLMs 스타일 처리 |

## 📊 **Window & Chunk 메커니즘**
- **Window Size**: 16 프레임 (과거 컨텍스트)
- **Chunk Size**: 2 프레임 (미래 예측)
- **Total Sequence**: 18 프레임 (16+2)
- **예측 목표**: 현재 이미지로 다음 2 프레임 액션 예측


In [6]:
# Cell 1: 🔧 환경 설정 및 라이브러리 임포트

import os
import sys
import json
import torch
import numpy as np
from pathlib import Path
from datetime import datetime
from transformers import AutoProcessor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 프로젝트 루트 디렉토리 설정
ROOT_DIR = Path("/home/billy/25-1kp/vla/Robo+/Mobile_VLA")
DATA_DIR = Path("/home/billy/25-1kp/vla/ROS_action/mobile_vla_dataset")  # 실제 데이터셋 경로
ROBOVLMS_DIR = Path("/home/billy/25-1kp/vla/RoboVLMs")

# 패키지 경로 추가
sys.path.append(str(ROOT_DIR))
sys.path.append(str(ROBOVLMS_DIR))

print(f"📁 작업 디렉토리: {ROOT_DIR}")
print(f"📊 실제 데이터 디렉토리: {DATA_DIR}")
print(f"🤖 RoboVLMs 디렉토리: {ROBOVLMS_DIR}")

# 데이터 디렉토리 존재 확인
if DATA_DIR.exists():
    h5_files = list(DATA_DIR.glob("*.h5"))
    print(f"✅ 데이터 디렉토리 확인됨: {len(h5_files)}개 H5 파일 발견")
else:
    print(f"❌ 데이터 디렉토리를 찾을 수 없습니다: {DATA_DIR}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 사용 디바이스: {device}")


📁 작업 디렉토리: /home/billy/25-1kp/vla/Robo+/Mobile_VLA
📊 실제 데이터 디렉토리: /home/billy/25-1kp/vla/ROS_action/mobile_vla_dataset
🤖 RoboVLMs 디렉토리: /home/billy/25-1kp/vla/RoboVLMs
✅ 데이터 디렉토리 확인됨: 72개 H5 파일 발견
🖥️ 사용 디바이스: cuda


In [7]:
# Cell 2: 📦 핵심 모듈 로드 (Window/Chunk 처리 포함)

def load_class_from_file(class_name, file_path, target_class):
    """파일에서 클래스를 동적으로 로드"""
    import importlib.util
    spec = importlib.util.spec_from_file_location(class_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return getattr(module, target_class)

# 업데이트된 모듈들 로드
WindowChunkAdapter = load_class_from_file(
    'window_chunk_adapter', 
    str(ROOT_DIR / 'data' / 'window_chunk_adapter.py'), 
    'WindowChunkAdapter'
)

ActionPredictionTrainer = load_class_from_file(
    'action_trainer', 
    str(ROOT_DIR / 'training' / 'action_trainer.py'), 
    'ActionPredictionTrainer'
)

# Kosmos 프로세서 초기화
kosmos_processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")
print("✅ 모든 모듈 로드 완료!")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ 모든 모듈 로드 완료!


In [8]:
# Cell 3: 🗂️ Window/Chunk 데이터셋 초기화 (업데이트된 시나리오 매핑)

# 먼저 업데이트된 WindowChunkAdapter 다시 로드
print("🔄 업데이트된 모듈 다시 로드 중...")
import importlib
import sys

# 모듈 다시 로드
if 'window_chunk_adapter' in sys.modules:
    importlib.reload(sys.modules['window_chunk_adapter'])

WindowChunkAdapter = load_class_from_file(
    'window_chunk_adapter', 
    str(ROOT_DIR / 'data' / 'window_chunk_adapter.py'), 
    'WindowChunkAdapter'
)

# 액션 예측 중심 데이터셋 초기화
print("📊 데이터셋 초기화 중...")
dataset = WindowChunkAdapter(
    data_dir=str(DATA_DIR),
    window_size=16,  # 과거 컨텍스트
    chunk_size=2,    # 미래 예측
    image_processor=kosmos_processor.image_processor,
    normalize_actions=True
)

print(f"\n📊 액션 예측 데이터셋 정보:")
print(f"   총 샘플: {len(dataset)}개")
print(f"   Window Size: {dataset.window_size} 프레임")
print(f"   Chunk Size: {dataset.chunk_size} 프레임")
print(f"   Total Sequence: {dataset.window_size + dataset.chunk_size} 프레임")

if len(dataset) > 0:
    # 데이터 로더 설정
    dataloader = DataLoader(
        dataset, 
        batch_size=4, 
        shuffle=True, 
        num_workers=2
    )
    
    print(f"   배치 크기: {dataloader.batch_size}")
    print(f"   총 배치 수: {len(dataloader)}")
    
    # 시나리오 분포 확인
    scenarios = [dataset.samples[i]['scenario'] for i in range(len(dataset))]
    from collections import Counter
    scenario_counts = Counter(scenarios)
    
    print(f"\n🎯 시나리오별 분포:")
    for scenario, count in scenario_counts.items():
        print(f"   {scenario}: {count}개")
        
else:
    print("❌ 유효한 데이터가 없습니다. 시나리오 매핑을 확인해주세요.")


🔄 업데이트된 모듈 다시 로드 중...
📊 데이터셋 초기화 중...

📊 액션 예측 데이터셋 정보:
   총 샘플: 72개
   Window Size: 16 프레임
   Chunk Size: 2 프레임
   Total Sequence: 18 프레임
   배치 크기: 4
   총 배치 수: 18

🎯 시나리오별 분포:
   2box_left_vertical: 10개
   2box_right_horizontal: 6개
   1box_left_vertical: 10개
   1box_right_horizontal: 10개
   2box_right_vertical: 10개
   1box_left_horizontal: 10개
   1box_right_vertical: 10개
   2box_left_horizontal: 6개


In [9]:
# Cell 4: 🔍 샘플 데이터 구조 분석

# 첫 번째 배치로 데이터 구조 확인
sample_batch = next(iter(dataloader))

print("🔍 샘플 배치 데이터 구조:")
print("=" * 50)

for key, value in sample_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"📊 {key}: {value.shape} ({value.dtype})")
    else:
        print(f"📝 {key}: {type(value)} - {len(value) if hasattr(value, '__len__') else 'scalar'}")

print("\n🎯 액션 예측 타겟:")
print(f"   입력 이미지: {sample_batch['pixel_values'].shape} (Window)")
print(f"   예측 액션: {sample_batch['target_actions'].shape} (Chunk)")
print(f"   액션 범위: [{sample_batch['target_actions'].min().item():.3f}, {sample_batch['target_actions'].max().item():.3f}]")


🔍 샘플 배치 데이터 구조:
📊 pixel_values: torch.Size([4, 16, 3, 224, 224]) (torch.float32)
📊 target_actions: torch.Size([4, 2, 3]) (torch.float32)
📊 context_actions: torch.Size([4, 16, 3]) (torch.float32)
📝 task_description: <class 'list'> - 4
📝 scenario: <class 'list'> - 4
📝 episode_id: <class 'list'> - 4

🎯 액션 예측 타겟:
   입력 이미지: torch.Size([4, 16, 3, 224, 224]) (Window)
   예측 액션: torch.Size([4, 2, 3]) (Chunk)
   액션 범위: [-1.000, 1.000]


In [10]:
# Cell 5: 🤖 액션 예측 트레이너 초기화

# 액션 예측 트레이너 초기화
trainer = ActionPredictionTrainer(
    model_name="microsoft/kosmos-2-patch14-224",
    action_dim=3,  # [linear_x, linear_y, angular_z]
    window_size=16,
    chunk_size=2,
    learning_rate=1e-4,
    device=device
)

print("🤖 액션 예측 모델 구성:")
print("=" * 40)
print(f"📝 VLM 백본: Kosmos 2B")
print(f"🎯 액션 차원: 3D (Mobile Navigation)")
print(f"🪟 Window Size: {trainer.window_size}")
print(f"🧩 Chunk Size: {trainer.chunk_size}")
print(f"📈 Learning Rate: {trainer.learning_rate}")
print(f"💾 모델 파라미터: {sum(p.numel() for p in trainer.model.parameters() if p.requires_grad):,}")

print("✅ 액션 예측 시스템 초기화 완료!")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


🤖 액션 예측 모델 구성:
📝 VLM 백본: Kosmos 2B
🎯 액션 차원: 3D (Mobile Navigation)
🪟 Window Size: 16
🧩 Chunk Size: 2
📈 Learning Rate: 0.0001
💾 모델 파라미터: 1,669,340,422
✅ 액션 예측 시스템 초기화 완료!


In [11]:
# Cell 6: 🚀 액션 예측 학습 루프

def train_action_prediction(
    trainer, 
    dataloader, 
    num_epochs=10,
    save_interval=5,
    log_interval=10
):
    """액션 예측 중심 학습 함수"""
    
    # 손실 추적기
    class ActionLossTracker:
        def __init__(self):
            self.losses = {
                'steps': [],
                'action_loss': [],
                'total_loss': [],
                'lr': [],
                'mae_linear_x': [],
                'mae_linear_y': [], 
                'mae_angular_z': [],
                'mae_avg': [],
                'scenarios': []
            }
        
        def update(self, step, loss_dict, scenario=None):
            self.losses['steps'].append(step)
            for key in ['action_loss', 'total_loss', 'lr', 'mae_linear_x', 'mae_linear_y', 'mae_angular_z', 'mae_avg']:
                if key in loss_dict:
                    self.losses[key].append(loss_dict[key])
                else:
                    self.losses[key].append(0.0)
            self.losses['scenarios'].append(scenario or 'unknown')
    
    loss_tracker = ActionLossTracker()
    best_loss = float('inf')
    global_step = 0
    
    print("🚀 액션 예측 학습 시작!")
    print("=" * 60)
    
    for epoch in range(num_epochs):
        epoch_losses = []
        
        # 에포크 진행 바
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(pbar):
            try:
                # 학습 스텝
                loss_dict = trainer.train_step(batch)
                epoch_losses.append(loss_dict['total_loss'])
                global_step += 1
                
                # 손실 추적
                scenario = batch.get('scenario', ['unknown'])[0] if 'scenario' in batch else 'unknown'
                loss_tracker.update(global_step, loss_dict, scenario)
                
                # 로그 출력
                if global_step % log_interval == 0:
                    pbar.set_postfix({
                        'Loss': f"{loss_dict['total_loss']:.4f}",
                        'MAE_x': f"{loss_dict.get('mae_linear_x', 0):.4f}",
                        'MAE_y': f"{loss_dict.get('mae_linear_y', 0):.4f}",
                        'MAE_z': f"{loss_dict.get('mae_angular_z', 0):.4f}",
                        'LR': f"{loss_dict['lr']:.2e}"
                    })
                
            except Exception as e:
                print(f"\n❌ 학습 스텝 실패 (Step {global_step}): {e}")
                continue
        
        # 에포크 요약
        avg_loss = np.mean(epoch_losses) if epoch_losses else float('inf')
        print(f"\n📊 Epoch {epoch+1} 완료:")
        print(f"   평균 손실: {avg_loss:.4f}")
        print(f"   처리된 배치: {len(epoch_losses)}/{len(dataloader)}")
        
        # 체크포인트 저장
        if avg_loss < best_loss:
            best_loss = avg_loss
            save_path = ROOT_DIR / f"checkpoints/action_prediction_best.pth"
            save_path.parent.mkdir(exist_ok=True)
            trainer.save_checkpoint(str(save_path), epoch, best_loss)
            print(f"   ✅ 최고 모델 저장: {save_path}")
        
        if (epoch + 1) % save_interval == 0:
            save_path = ROOT_DIR / f"checkpoints/action_prediction_epoch_{epoch+1}.pth"
            save_path.parent.mkdir(exist_ok=True)
            trainer.save_checkpoint(str(save_path), epoch, best_loss)
            print(f"   💾 체크포인트 저장: {save_path}")
    
    print("\n🎉 학습 완료!")
    print(f"최종 최고 손실: {best_loss:.4f}")
    
    return loss_tracker

print("⚡ 액션 예측 학습 준비 완료!")


⚡ 액션 예측 학습 준비 완료!


In [ ]:
# # Cell 7: 🎯 학습 실행

# print("⚡ 액션 예측 학습을 시작합니다...")
# final_loss_tracker = train_action_prediction(
#     trainer=trainer,
#     dataloader=dataloader, 
#     num_epochs=15,
#     save_interval=5,
#     log_interval=5
# )


In [ ]:
# Cell 8: 📊 액션 예측 평가 및 분석

def evaluate_action_prediction(trainer, dataloader, num_samples=20):
    """액션 예측 성능 평가"""
    
    print("🔍 액션 예측 성능 평가 시작...")
    
    all_predictions = []
    all_targets = []
    all_scenarios = []
    
    trainer.model.eval()
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader, desc="평가 중")):
            if i >= num_samples:
                break
            
            try:
                # 평가 스텝
                results = trainer.evaluate_step(batch)
                
                # 결과 수집
                pred_actions = results['predictions']['predicted_actions']  # [B, T, chunk_size, action_dim]
                target_actions = results['targets']['target_actions']  # [B, chunk_size, action_dim]
                scenarios = results['scenarios']
                
                # Batch dimension으로 펼치기
                for b in range(pred_actions.shape[0]):
                    # 마지막 시간 스텝의 예측만 사용 (가장 관련성 높음)
                    pred = pred_actions[b, -1, :, :].cpu()  # [chunk_size, action_dim]
                    target = target_actions[b, :, :].cpu()  # [chunk_size, action_dim]
                    
                    all_predictions.append(pred)
                    all_targets.append(target)
                    all_scenarios.append(scenarios[b] if b < len(scenarios) else 'unknown')
                    
            except Exception as e:
                print(f"평가 중 오류: {e}")
                continue
    
    if not all_predictions:
        print("❌ 평가할 데이터가 없습니다.")
        return None
    
    # 텐서로 변환
    all_predictions = torch.stack(all_predictions)  # [N, chunk_size, action_dim]
    all_targets = torch.stack(all_targets)  # [N, chunk_size, action_dim]
    
    print(f"\n📊 평가 완료: {len(all_predictions)}개 샘플")
    print(f"예측 shape: {all_predictions.shape}")
    print(f"타겟 shape: {all_targets.shape}")
    
    # 성능 메트릭 계산
    metrics = {}
    
    # 전체 MAE
    mae_total = torch.abs(all_predictions - all_targets).mean()
    metrics['mae_total'] = mae_total.item()
    
    # 차원별 MAE
    mae_per_dim = torch.abs(all_predictions - all_targets).mean(dim=(0, 1))  # [action_dim]
    dim_names = ['linear_x', 'linear_y', 'angular_z']
    for i, name in enumerate(dim_names):
        metrics[f'mae_{name}'] = mae_per_dim[i].item()
    
    # MSE
    mse_total = ((all_predictions - all_targets) ** 2).mean()
    metrics['mse_total'] = mse_total.item()
    
    # RMSE
    metrics['rmse_total'] = torch.sqrt(mse_total).item()
    
    # R² Score (per dimension)
    for i, name in enumerate(dim_names):
        pred_dim = all_predictions[:, :, i].flatten()
        target_dim = all_targets[:, :, i].flatten()
        
        ss_res = ((target_dim - pred_dim) ** 2).sum()
        ss_tot = ((target_dim - target_dim.mean()) ** 2).sum()
        r2 = 1 - (ss_res / ss_tot)
        metrics[f'r2_{name}'] = r2.item()
    
    # 시나리오별 성능
    scenario_metrics = {}
    unique_scenarios = list(set(all_scenarios))
    
    for scenario in unique_scenarios:
        scenario_mask = [i for i, s in enumerate(all_scenarios) if s == scenario]
        if len(scenario_mask) > 0:
            scenario_preds = all_predictions[scenario_mask]
            scenario_targets = all_targets[scenario_mask]
            scenario_mae = torch.abs(scenario_preds - scenario_targets).mean()
            scenario_metrics[scenario] = scenario_mae.item()
    
    return {
        'predictions': all_predictions,
        'targets': all_targets,
        'scenarios': all_scenarios,
        'metrics': metrics,
        'scenario_metrics': scenario_metrics
    }

# 평가 실행
if 'final_loss_tracker' in locals():
    print("🔍 학습된 모델 평가 시작...")
    eval_results = evaluate_action_prediction(trainer, dataloader, num_samples=30)
    
    if eval_results:
        print("\n📈 액션 예측 성능 결과:")
        print("=" * 50)
        for key, value in eval_results['metrics'].items():
            print(f"  {key}: {value:.4f}")
        
        print("\n🎯 시나리오별 MAE:")
        for scenario, mae in eval_results['scenario_metrics'].items():
            print(f"  {scenario}: {mae:.4f}")
else:
    print("⚠️ 먼저 학습을 완료해주세요 (Cell 7 실행)")


In [ ]:
# Cell 9: 📈 결과 시각화 및 분석

def plot_action_prediction_results(loss_tracker, eval_results=None):
    """액션 예측 결과 시각화"""
    
    if not hasattr(loss_tracker, 'losses') or not loss_tracker.losses['steps']:
        print("❌ 시각화할 학습 데이터가 없습니다.")
        return
    
    # 플롯 설정
    plt.style.use('default')
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('🚀 Mobile VLA 액션 예측 학습 결과', fontsize=16, fontweight='bold')
    
    # 1. 총 손실 곡선
    axes[0, 0].plot(loss_tracker.losses['steps'], loss_tracker.losses['total_loss'], 'b-', linewidth=2)
    axes[0, 0].set_title('📊 Total Loss', fontweight='bold')
    axes[0, 0].set_xlabel('Steps')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 차원별 MAE
    if 'mae_linear_x' in loss_tracker.losses:
        axes[0, 1].plot(loss_tracker.losses['steps'], loss_tracker.losses['mae_linear_x'], 'r-', label='linear_x', linewidth=2)
        axes[0, 1].plot(loss_tracker.losses['steps'], loss_tracker.losses['mae_linear_y'], 'g-', label='linear_y', linewidth=2)
        axes[0, 1].plot(loss_tracker.losses['steps'], loss_tracker.losses['mae_angular_z'], 'b-', label='angular_z', linewidth=2)
        axes[0, 1].set_title('🎯 MAE per Dimension', fontweight='bold')
        axes[0, 1].set_xlabel('Steps')
        axes[0, 1].set_ylabel('MAE')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 학습률
    axes[0, 2].plot(loss_tracker.losses['steps'], loss_tracker.losses['lr'], 'purple', linewidth=2)
    axes[0, 2].set_title('📈 Learning Rate', fontweight='bold')
    axes[0, 2].set_xlabel('Steps')
    axes[0, 2].set_ylabel('LR')
    axes[0, 2].set_yscale('log')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. 평가 결과 (있는 경우)
    if eval_results and 'metrics' in eval_results:
        metrics = eval_results['metrics']
        
        # MAE 비교
        mae_names = ['mae_linear_x', 'mae_linear_y', 'mae_angular_z']
        mae_values = [metrics.get(name, 0) for name in mae_names]
        mae_labels = ['Linear X', 'Linear Y', 'Angular Z']
        
        bars = axes[1, 0].bar(mae_labels, mae_values, color=['red', 'green', 'blue'], alpha=0.7)
        axes[1, 0].set_title('🎯 Final MAE by Dimension', fontweight='bold')
        axes[1, 0].set_ylabel('MAE')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 막대 위에 값 표시
        for bar, value in zip(bars, mae_values):
            axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                           f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # R² Score
        r2_names = ['r2_linear_x', 'r2_linear_y', 'r2_angular_z']
        r2_values = [metrics.get(name, 0) for name in r2_names]
        
        bars_r2 = axes[1, 1].bar(mae_labels, r2_values, color=['red', 'green', 'blue'], alpha=0.7)
        axes[1, 1].set_title('📊 R² Score by Dimension', fontweight='bold')
        axes[1, 1].set_ylabel('R² Score')
        axes[1, 1].set_ylim(0, 1)
        axes[1, 1].grid(True, alpha=0.3)
        
        # R² 값 표시
        for bar, value in zip(bars_r2, r2_values):
            axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                           f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 시나리오별 성능
        if 'scenario_metrics' in eval_results:
            scenario_data = eval_results['scenario_metrics']
            scenarios = list(scenario_data.keys())
            scenario_maes = list(scenario_data.values())
            
            bars_scenario = axes[1, 2].bar(range(len(scenarios)), scenario_maes, 
                                         color='orange', alpha=0.7)
            axes[1, 2].set_title('🎮 Performance by Scenario', fontweight='bold')
            axes[1, 2].set_xlabel('Scenario')
            axes[1, 2].set_ylabel('MAE')
            axes[1, 2].set_xticks(range(len(scenarios)))
            axes[1, 2].set_xticklabels([s.replace('_', '\n') for s in scenarios], 
                                      rotation=45, ha='right', fontsize=8)
            axes[1, 2].grid(True, alpha=0.3)
    else:
        # 평가 결과가 없으면 빈 플롯
        for i in range(3):
            axes[1, i].text(0.5, 0.5, '평가 결과가 없습니다\n(Cell 8 실행 필요)', 
                           ha='center', va='center', transform=axes[1, i].transAxes,
                           fontsize=12, style='italic')
            axes[1, i].set_xticks([])
            axes[1, i].set_yticks([])
    
    plt.tight_layout()
    plt.show()
    
    # 텍스트 요약
    print("\n📋 학습 요약:")
    print("=" * 50)
    if loss_tracker.losses['total_loss']:
        print(f"🔹 최종 손실: {loss_tracker.losses['total_loss'][-1]:.4f}")
        print(f"🔹 최저 손실: {min(loss_tracker.losses['total_loss']):.4f}")
        print(f"🔹 총 스텝: {len(loss_tracker.losses['steps'])}")
    
    if eval_results and 'metrics' in eval_results:
        print(f"🔹 평가 MAE: {eval_results['metrics']['mae_total']:.4f}")
        print(f"🔹 평가 RMSE: {eval_results['metrics']['rmse_total']:.4f}")

# 시각화 실행
if 'final_loss_tracker' in locals():
    plot_action_prediction_results(
        final_loss_tracker, 
        eval_results if 'eval_results' in locals() else None
    )
else:
    print("⚠️ 먼저 학습을 완료해주세요 (Cell 7 실행)")


In [ ]:
# Cell 10: 💾 결과 저장 및 리포트 생성

def save_action_prediction_results(loss_tracker, eval_results, save_dir=None):
    """액션 예측 결과를 마크다운과 JSON으로 저장"""
    
    if save_dir is None:
        save_dir = ROOT_DIR / "action_prediction_results"
    else:
        save_dir = Path(save_dir)
    
    save_dir.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 마크다운 리포트 생성
    md_content = f"""# 🚀 Mobile VLA 액션 예측 학습 리포트

**생성일시**: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
**시스템**: 액션 예측 중심 Mobile VLA + Kosmos 2B

## 📊 **핵심 개선사항**

| 항목 | 이전 (이벤트 분류) | 현재 (액션 예측) | 개선 효과 |
|------|------------------|------------------|-----------|
| **학습 목표** | 🔴 이벤트 99.9% (너무 단순) | ✅ 이미지→액션 매핑 | 진짜 VLA 학습 |
| **메커니즘** | 🔴 Frame-by-frame | ✅ Window/Chunk | RoboVLMs 스타일 |
| **손실 함수** | 🔴 CrossEntropy + MSE | ✅ Huber Loss | 회귀 중심 |
| **메트릭** | 🔴 Accuracy, F1-Score | ✅ MAE, R², RMSE | 연속 액션 평가 |

## 🎯 **학습 설정**

- **Window Size**: 16 프레임 (과거 컨텍스트)
- **Chunk Size**: 2 프레임 (미래 예측)
- **액션 차원**: 3D [linear_x, linear_y, angular_z]
- **모델**: Kosmos 2B + ActionPredictionHead
- **손실**: Huber Loss (outlier robust)

## 📈 **학습 결과**
"""
    
    if hasattr(loss_tracker, 'losses') and loss_tracker.losses['steps']:
        final_loss = loss_tracker.losses['total_loss'][-1]
        min_loss = min(loss_tracker.losses['total_loss'])
        total_steps = len(loss_tracker.losses['steps'])
        
        md_content += f"""
### 🏋️ **학습 성능**

- **최종 손실**: {final_loss:.4f}
- **최저 손실**: {min_loss:.4f}  
- **총 학습 스텝**: {total_steps:,}
- **수렴성**: {'✅ 안정적' if final_loss < min_loss * 1.1 else '⚠️ 추가 학습 필요'}
"""
    
    if eval_results and 'metrics' in eval_results:
        metrics = eval_results['metrics']
        md_content += f"""
### 🎯 **평가 성능**

#### 전체 메트릭
- **Total MAE**: {metrics['mae_total']:.4f}
- **Total MSE**: {metrics['mse_total']:.4f}
- **Total RMSE**: {metrics['rmse_total']:.4f}

#### 차원별 성능
| 액션 차원 | MAE | R² Score | 해석 |
|----------|-----|----------|------|
| **Linear X** | {metrics.get('mae_linear_x', 0):.4f} | {metrics.get('r2_linear_x', 0):.3f} | {'✅ 우수' if metrics.get('mae_linear_x', 1) < 0.1 else '⚠️ 개선 필요'} |
| **Linear Y** | {metrics.get('mae_linear_y', 0):.4f} | {metrics.get('r2_linear_y', 0):.3f} | {'✅ 우수' if metrics.get('mae_linear_y', 1) < 0.1 else '⚠️ 개선 필요'} |
| **Angular Z** | {metrics.get('mae_angular_z', 0):.4f} | {metrics.get('r2_angular_z', 0):.3f} | {'✅ 우수' if metrics.get('mae_angular_z', 1) < 0.1 else '⚠️ 개선 필요'} |
"""
        
        if 'scenario_metrics' in eval_results:
            md_content += """
#### 시나리오별 성능
| 시나리오 | MAE | 상대 성능 |
|----------|-----|-----------|
"""
            scenario_data = eval_results['scenario_metrics']
            avg_mae = metrics['mae_total']
            for scenario, mae in scenario_data.items():
                relative_perf = "✅ 우수" if mae < avg_mae else "⚠️ 평균 이하"
                md_content += f"| {scenario} | {mae:.4f} | {relative_perf} |\n"
    
    md_content += f"""
## 🔄 **이전 vs 현재 비교**

### 학습 패러다임 변화
1. **분류 → 회귀**: 이벤트 분류에서 액션 예측으로 전환
2. **단순 → 복잡**: 99.9% 정확도에서 진짜 어려운 문제로
3. **프레임 → 시퀀스**: Window/Chunk 메커니즘으로 시간적 관계 학습
4. **수동 → 자동**: 수동 특징에서 end-to-end 학습으로

### 기대 효과
- **실제 로봇 적용**: 이미지 보고 실시간 액션 생성 가능
- **일반화 성능**: 다양한 장애물 패턴에서 robust한 네비게이션
- **확장성**: 다른 모바일 로봇 태스크로 쉽게 확장 가능

## 🚀 **다음 단계**

1. **실시간 테스트**: 실제 로봇에서 inference 성능 검증
2. **데이터 증강**: 더 다양한 시나리오와 환경 조건 추가
3. **모델 최적화**: TensorRT, ONNX 등으로 추론 속도 개선
4. **멀티태스크**: 컵 추적 외 다른 네비게이션 태스크 추가

---
**생성 시스템**: Mobile VLA + Kosmos 2B Action Prediction
**보고서 버전**: v2.0 (액션 예측 중심)
"""
    
    # 마크다운 파일 저장
    md_path = save_dir / f"Action_Prediction_Report_{timestamp}.md"
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(md_content)
    
    # JSON 데이터 저장
    json_data = {
        'timestamp': timestamp,
        'system_info': {
            'model': 'Kosmos 2B + ActionPredictionHead',
            'window_size': 16,
            'chunk_size': 2,
            'action_dim': 3,
            'loss_function': 'Huber Loss'
        },
        'training_results': {},
        'evaluation_results': {}
    }
    
    # 학습 결과 추가
    if hasattr(loss_tracker, 'losses'):
        json_data['training_results'] = {
            'final_loss': loss_tracker.losses['total_loss'][-1] if loss_tracker.losses['total_loss'] else None,
            'min_loss': min(loss_tracker.losses['total_loss']) if loss_tracker.losses['total_loss'] else None,
            'total_steps': len(loss_tracker.losses['steps']),
            'loss_history': loss_tracker.losses['total_loss'][-100:],  # 마지막 100개만 저장
            'mae_history': {
                'linear_x': loss_tracker.losses['mae_linear_x'][-100:] if 'mae_linear_x' in loss_tracker.losses else [],
                'linear_y': loss_tracker.losses['mae_linear_y'][-100:] if 'mae_linear_y' in loss_tracker.losses else [],
                'angular_z': loss_tracker.losses['mae_angular_z'][-100:] if 'mae_angular_z' in loss_tracker.losses else []
            }
        }
    
    # 평가 결과 추가
    if eval_results:
        json_data['evaluation_results'] = {
            'metrics': eval_results.get('metrics', {}),
            'scenario_metrics': eval_results.get('scenario_metrics', {}),
            'num_samples': len(eval_results.get('predictions', []))
        }
    
    # JSON 파일 저장
    json_path = save_dir / f"Action_Prediction_Data_{timestamp}.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ 결과 저장 완료:")
    print(f"   📄 마크다운: {md_path}")
    print(f"   📊 JSON: {json_path}")
    
    return md_path, json_path

# 결과 저장 실행
if 'final_loss_tracker' in locals():
    print("💾 액션 예측 결과를 저장합니다...")
    md_path, json_path = save_action_prediction_results(
        final_loss_tracker,
        eval_results if 'eval_results' in locals() else None
    )
    print("🎉 모든 결과가 성공적으로 저장되었습니다!")
else:
    print("⚠️ 먼저 학습을 완료해주세요 (Cell 7 실행)")


⚠️ 먼저 학습을 완료해주세요 (Cell 7 실행)


In [ ]:
# Cell 11: 🧪 빠른 테스트 및 문제 해결

# 데이터셋이 제대로 로드되었는지 확인
if 'dataset' in locals() and len(dataset) > 0:
    print("✅ 데이터셋 로드 성공!")
    
    # 첫 번째 샘플 테스트
    try:
        sample = dataset[0]
        print(f"\n📊 첫 번째 샘플 확인:")
        for key, value in sample.items():
            if isinstance(value, torch.Tensor):
                print(f"   {key}: {value.shape} ({value.dtype})")
            else:
                print(f"   {key}: {type(value)}")
        
        print(f"\n🎯 태스크: {sample['task_description']}")
        print(f"🎮 시나리오: {sample['scenario']}")
        
    except Exception as e:
        print(f"❌ 샘플 로드 오류: {e}")
        import traceback
        traceback.print_exc()

elif 'dataset' in locals() and len(dataset) == 0:
    print("⚠️ 데이터셋이 비어있습니다.")
    print("\n🔍 문제 해결 방법:")
    print("1. 파일 경로 확인: /home/billy/25-1kp/vla/ROS_action/mobile_vla_dataset/")
    print("2. HDF5 파일 존재 확인")
    print("3. 시나리오 매핑 로직 확인")
    
    # 데이터 디렉토리 내용 확인
    print(f"\n📁 데이터 디렉토리 내용:")
    h5_files = list(DATA_DIR.glob("*.h5"))
    print(f"   총 HDF5 파일: {len(h5_files)}개")
    
    if h5_files:
        print(f"   예시 파일: {h5_files[0].name}")
        
        # 시나리오 추출 테스트
        test_filename = h5_files[0].name
        print(f"\n🧪 시나리오 추출 테스트: {test_filename}")
        
        # 수동으로 시나리오 추출 로직 테스트
        filename_lower = test_filename.lower()
        print(f"   소문자 변환: {filename_lower}")
        
        if "1box" in filename_lower and "vert" in filename_lower and "left" in filename_lower:
            result = "1box_left_vertical"
        elif "1box" in filename_lower and "hori" in filename_lower and "left" in filename_lower:
            result = "1box_left_horizontal"
        else:
            result = "unknown"
        
        print(f"   추출된 시나리오: {result}")

else:
    print("❌ 데이터셋이 정의되지 않았습니다. Cell 3을 먼저 실행해주세요.")


In [14]:
# Cell 12: 🧹 메모리 최적화 및 정리

import gc
import torch

def clear_gpu_memory():
    """GPU 메모리 정리"""
    print("🧹 GPU 메모리 정리 중...")
    
    # 캐시 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
    # 가비지 컬렉션
    gc.collect()
    
    # 메모리 사용량 출력
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        print(f"   할당된 메모리: {allocated:.2f} GB")
        print(f"   캐시된 메모리: {cached:.2f} GB")

# 메모리 정리 실행
clear_gpu_memory()

# 메모리 효율적인 설정으로 데이터셋 및 트레이너 재초기화
print("\n⚙️ 메모리 효율적인 설정으로 재초기화...")

# 1. 배치 크기 줄이기
if 'dataloader' in locals():
    del dataloader

# 더 작은 배치 크기로 데이터로더 재생성
dataloader = DataLoader(
    dataset, 
    batch_size=1,  # 4 -> 1로 줄임
    shuffle=True, 
    num_workers=0  # 메모리 절약을 위해 0으로 설정
)

print(f"   배치 크기: {dataloader.batch_size} (메모리 절약)")
print(f"   총 배치 수: {len(dataloader)}")

# 2. 트레이너 재초기화 (더 작은 윈도우 사이즈)
if 'trainer' in locals():
    del trainer

clear_gpu_memory()

# 메모리 효율적인 트레이너
memory_efficient_trainer = ActionPredictionTrainer(
    model_name="microsoft/kosmos-2-patch14-224",
    action_dim=3,
    window_size=8,   # 16 -> 8로 줄임
    chunk_size=2,
    learning_rate=1e-4,
    device=device
)

print(f"   윈도우 크기: {memory_efficient_trainer.window_size} (메모리 절약)")
print(f"   청크 크기: {memory_efficient_trainer.chunk_size}")

# 메모리 사용량 재확인
clear_gpu_memory()


🧹 GPU 메모리 정리 중...
   할당된 메모리: 6.24 GB
   캐시된 메모리: 6.53 GB

⚙️ 메모리 효율적인 설정으로 재초기화...
   배치 크기: 1 (메모리 절약)
   총 배치 수: 72
🧹 GPU 메모리 정리 중...
   할당된 메모리: 0.01 GB
   캐시된 메모리: 0.03 GB


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


   윈도우 크기: 8 (메모리 절약)
   청크 크기: 2
🧹 GPU 메모리 정리 중...
   할당된 메모리: 6.24 GB
   캐시된 메모리: 6.26 GB


In [16]:
# Cell 13: 🔧 메모리 효율적인 데이터셋 재생성

# 윈도우 크기에 맞춰 데이터셋 재생성
print("🔧 메모리 효율적인 데이터셋 재생성 중...")

# 기존 데이터셋 정리
if 'dataset' in locals():
    del dataset

clear_gpu_memory()

# 메모리 효율적인 데이터셋 (윈도우 8 + 청크 2 = 10프레임)
memory_efficient_dataset = WindowChunkAdapter(
    data_dir=str(DATA_DIR),
    window_size=8,   # 16 -> 8로 줄임
    chunk_size=2,
    image_processor=kosmos_processor.image_processor,
    normalize_actions=True
)

print(f"\n📊 메모리 효율적인 데이터셋 정보:")
print(f"   총 샘플: {len(memory_efficient_dataset)}개")
print(f"   Window Size: {memory_efficient_dataset.window_size} 프레임")
print(f"   Chunk Size: {memory_efficient_dataset.chunk_size} 프레임")
print(f"   Total Sequence: {memory_efficient_dataset.window_size + memory_efficient_dataset.chunk_size} 프레임")

if len(memory_efficient_dataset) > 0:
    # 메모리 효율적인 데이터로더
    memory_efficient_dataloader = DataLoader(
        memory_efficient_dataset, 
        batch_size=1,  # 최소 배치 크기
        shuffle=True, 
        num_workers=0
    )
    
    print(f"   배치 크기: {memory_efficient_dataloader.batch_size}")
    print(f"   총 배치 수: {len(memory_efficient_dataloader)}")
    
    # 시나리오 분포 확인
    scenarios = [memory_efficient_dataset.samples[i]['scenario'] for i in range(len(memory_efficient_dataset))]
    from collections import Counter
    scenario_counts = Counter(scenarios)
    
    print(f"\n🎯 시나리오별 분포:")
    for scenario, count in scenario_counts.items():
        print(f"   {scenario}: {count}개")
    
    # 샘플 테스트
    print(f"\n🧪 샘플 데이터 테스트:")
    sample = memory_efficient_dataset[0]
    for key, value in sample.items():
        if isinstance(value, torch.Tensor):
            print(f"   {key}: {value.shape} ({value.dtype})")
        else:
            print(f"   {key}: {type(value)}")
    
    clear_gpu_memory()
    print("\n✅ 메모리 효율적인 설정 완료!")
    
else:
    print("❌ 데이터셋이 비어있습니다.")


🔧 메모리 효율적인 데이터셋 재생성 중...
🧹 GPU 메모리 정리 중...
   할당된 메모리: 6.24 GB
   캐시된 메모리: 6.26 GB


유효한 샘플이 없습니다. 필터링 조건을 확인해주세요.



📊 메모리 효율적인 데이터셋 정보:
   총 샘플: 0개
   Window Size: 8 프레임
   Chunk Size: 2 프레임
   Total Sequence: 10 프레임
❌ 데이터셋이 비어있습니다.


In [17]:
# Cell 14: 🚀 메모리 효율적인 학습 실행

def train_memory_efficient(
    trainer, 
    dataloader, 
    num_epochs=5,  # 에포크 수 줄임
    save_interval=2,
    log_interval=10,
    memory_cleanup_interval=5  # 5 배치마다 메모리 정리
):
    """메모리 효율적인 학습 함수"""
    
    # 손실 추적기
    class ActionLossTracker:
        def __init__(self):
            self.losses = {
                'steps': [],
                'action_loss': [],
                'total_loss': [],
                'lr': [],
                'mae_linear_x': [],
                'mae_linear_y': [], 
                'mae_angular_z': [],
                'mae_avg': [],
                'scenarios': []
            }
        
        def update(self, step, loss_dict, scenario=None):
            self.losses['steps'].append(step)
            for key in ['action_loss', 'total_loss', 'lr', 'mae_linear_x', 'mae_linear_y', 'mae_angular_z', 'mae_avg']:
                if key in loss_dict:
                    self.losses[key].append(loss_dict[key])
                else:
                    self.losses[key].append(0.0)
            self.losses['scenarios'].append(scenario or 'unknown')
    
    loss_tracker = ActionLossTracker()
    best_loss = float('inf')
    global_step = 0
    
    print("🚀 메모리 효율적인 액션 예측 학습 시작!")
    print("=" * 60)
    print(f"📊 설정: 배치크기={dataloader.batch_size}, 윈도우={trainer.window_size}, 에포크={num_epochs}")
    
    for epoch in range(num_epochs):
        epoch_losses = []
        successful_batches = 0
        
        # 에포크 진행 바
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, batch in enumerate(pbar):
            try:
                # 학습 스텝
                loss_dict = trainer.train_step(batch)
                epoch_losses.append(loss_dict['total_loss'])
                global_step += 1
                successful_batches += 1
                
                # 손실 추적
                scenario = batch.get('scenario', ['unknown'])[0] if 'scenario' in batch else 'unknown'
                loss_tracker.update(global_step, loss_dict, scenario)
                
                # 로그 출력
                if global_step % log_interval == 0:
                    pbar.set_postfix({
                        'Loss': f"{loss_dict['total_loss']:.4f}",
                        'MAE_x': f"{loss_dict.get('mae_linear_x', 0):.4f}",
                        'MAE_y': f"{loss_dict.get('mae_linear_y', 0):.4f}",
                        'MAE_z': f"{loss_dict.get('mae_angular_z', 0):.4f}",
                        'LR': f"{loss_dict['lr']:.2e}",
                        'Success': f"{successful_batches}/{batch_idx+1}"
                    })
                
                # 메모리 정리
                if batch_idx % memory_cleanup_interval == 0:
                    clear_gpu_memory()
                
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"\\n⚠️ OOM 에러 (Step {global_step}), 메모리 정리 후 계속...")
                    clear_gpu_memory()
                    continue
                else:
                    print(f"\\n❌ 학습 스텝 실패 (Step {global_step}): {e}")
                    continue
            except Exception as e:
                print(f"\\n❌ 예상치 못한 오류 (Step {global_step}): {e}")
                continue
        
        # 에포크 요약
        avg_loss = np.mean(epoch_losses) if epoch_losses else float('inf')
        success_rate = successful_batches / len(dataloader) * 100
        
        print(f"\\n📊 Epoch {epoch+1} 완료:")
        print(f"   평균 손실: {avg_loss:.4f}")
        print(f"   성공률: {success_rate:.1f}% ({successful_batches}/{len(dataloader)})")
        
        # 체크포인트 저장
        if avg_loss < best_loss and epoch_losses:
            best_loss = avg_loss
            save_path = ROOT_DIR / f"checkpoints/memory_efficient_best.pth"
            save_path.parent.mkdir(exist_ok=True)
            trainer.save_checkpoint(str(save_path), epoch, best_loss)
            print(f"   ✅ 최고 모델 저장: {save_path}")
        
        if (epoch + 1) % save_interval == 0:
            save_path = ROOT_DIR / f"checkpoints/memory_efficient_epoch_{epoch+1}.pth"
            save_path.parent.mkdir(exist_ok=True)
            trainer.save_checkpoint(str(save_path), epoch, best_loss)
            print(f"   💾 체크포인트 저장: {save_path}")
        
        # 에포크 끝에 메모리 정리
        clear_gpu_memory()
    
    print("\\n🎉 메모리 효율적인 학습 완료!")
    print(f"최종 최고 손실: {best_loss:.4f}")
    
    return loss_tracker

# 메모리 효율적인 학습 실행
if 'memory_efficient_trainer' in locals() and 'memory_efficient_dataloader' in locals():
    print("⚡ 메모리 효율적인 액션 예측 학습을 시작합니다...")
    print(f"🔧 메모리 설정: Window={memory_efficient_trainer.window_size}, Batch={memory_efficient_dataloader.batch_size}")
    
    memory_efficient_loss_tracker = train_memory_efficient(
        trainer=memory_efficient_trainer,
        dataloader=memory_efficient_dataloader, 
        num_epochs=5,  # 적은 에포크로 테스트
        save_interval=2,
        log_interval=5,
        memory_cleanup_interval=3
    )
else:
    print("❌ 먼저 Cell 12-13을 실행해주세요.")


❌ 먼저 Cell 12-13을 실행해주세요.


In [ ]:
# Cell 15: 🛠️ 테스트용 더미 데이터 생성

import h5py
import numpy as np
from PIL import Image

def create_dummy_mobile_vla_data(data_dir, num_episodes=10):
    """테스트용 더미 Mobile VLA 데이터 생성"""
    
    data_dir = Path(data_dir)
    data_dir.mkdir(parents=True, exist_ok=True)
    
    scenarios = [
        "1box_left_vertical", "1box_left_horizontal", 
        "1box_right_vertical", "1box_right_horizontal",
        "2box_left_vertical", "2box_left_horizontal",
        "2box_right_vertical", "2box_right_horizontal"
    ]
    
    print(f"🛠️ {data_dir}에 더미 데이터 생성 중...")
    
    for i in range(num_episodes):
        # 시나리오 선택
        scenario = scenarios[i % len(scenarios)]
        
        # 파일명 생성 (실제 패턴과 유사하게)
        filename = f"episode_20250820_{i:06d}_{scenario}_core_medium.h5"
        filepath = data_dir / filename
        
        # 더미 데이터 생성
        sequence_length = np.random.randint(50, 150)  # 50-150 프레임
        
        # 더미 이미지 (720x1280x3 -> Mobile VLA 원본 해상도)
        dummy_images = np.random.randint(0, 255, (sequence_length, 720, 1280, 3), dtype=np.uint8)
        
        # 더미 액션 ([linear_x, linear_y, angular_z])
        dummy_actions = np.random.randn(sequence_length, 3).astype(np.float32)
        dummy_actions[:, 0] = np.clip(dummy_actions[:, 0], -1.0, 1.0)  # linear_x
        dummy_actions[:, 1] = np.clip(dummy_actions[:, 1], -1.0, 1.0)  # linear_y  
        dummy_actions[:, 2] = np.clip(dummy_actions[:, 2], -2.0, 2.0)  # angular_z
        
        # 더미 이벤트 (0: episode_start, 1: start_action)
        dummy_events = np.ones(sequence_length, dtype=np.int32)
        dummy_events[0] = 0  # 첫 프레임은 episode_start
        
        # H5 파일로 저장
        with h5py.File(filepath, 'w') as f:
            f.create_dataset('images', data=dummy_images, compression='lzf')
            f.create_dataset('actions', data=dummy_actions)
            f.create_dataset('events', data=dummy_events)
            
            # 메타데이터
            f.attrs['scenario'] = scenario
            f.attrs['sequence_length'] = sequence_length
            f.attrs['created_by'] = 'mobile_vla_data_collector.py'
    
    print(f"✅ {num_episodes}개의 더미 에피소드 생성 완료!")
    print(f"   시나리오: {scenarios}")
    
    # 생성된 파일 확인
    created_files = list(data_dir.glob("*.h5"))
    print(f"   생성된 파일: {len(created_files)}개")
    for f in created_files[:3]:
        print(f"   - {f.name}")
    if len(created_files) > 3:
        print(f"   - ... (총 {len(created_files)}개)")

# 더미 데이터 생성
dummy_data_dir = ROOT_DIR / "dummy_data"
create_dummy_mobile_vla_data(dummy_data_dir, num_episodes=16)

# 데이터 디렉터리 업데이트
DATA_DIR = dummy_data_dir
print(f"\\n📊 데이터 디렉터리 업데이트: {DATA_DIR}")


In [ ]:
# Cell 16: 🔄 더미 데이터로 메모리 효율적인 데이터셋 재생성

# WindowChunkAdapter 모듈 리로드
import importlib
import sys
if 'window_chunk_adapter' in sys.modules:
    importlib.reload(sys.modules['window_chunk_adapter'])

# WindowChunkAdapter 다시 로드
WindowChunkAdapter = load_class_from_file(
    'window_chunk_adapter',
    str(ROOT_DIR / 'data' / 'window_chunk_adapter.py'),
    'WindowChunkAdapter'
)

print("🔄 더미 데이터로 메모리 효율적인 데이터셋 재생성 중...")

# 기존 데이터셋 정리
if 'memory_efficient_dataset' in locals():
    del memory_efficient_dataset

clear_gpu_memory()

# 더미 데이터로 메모리 효율적인 데이터셋 생성
memory_efficient_dataset = WindowChunkAdapter(
    data_dir=str(DATA_DIR),
    window_size=8,   # 16 -> 8로 줄임
    chunk_size=2,
    image_processor=kosmos_processor.image_processor,
    normalize_actions=True
)

print(f"\\n📊 더미 데이터 기반 메모리 효율적인 데이터셋 정보:")
print(f"   총 샘플: {len(memory_efficient_dataset)}개")
print(f"   Window Size: {memory_efficient_dataset.window_size} 프레임")
print(f"   Chunk Size: {memory_efficient_dataset.chunk_size} 프레임")
print(f"   Total Sequence: {memory_efficient_dataset.window_size + memory_efficient_dataset.chunk_size} 프레임")

if len(memory_efficient_dataset) > 0:
    # 메모리 효율적인 데이터로더
    memory_efficient_dataloader = DataLoader(
        memory_efficient_dataset, 
        batch_size=1,  # 최소 배치 크기
        shuffle=True, 
        num_workers=0
    )
    
    print(f"   배치 크기: {memory_efficient_dataloader.batch_size}")
    print(f"   총 배치 수: {len(memory_efficient_dataloader)}")
    
    # 시나리오 분포 확인
    scenarios = [memory_efficient_dataset.samples[i]['scenario'] for i in range(len(memory_efficient_dataset))]
    from collections import Counter
    scenario_counts = Counter(scenarios)
    
    print(f"\\n🎯 시나리오별 분포:")
    for scenario, count in scenario_counts.items():
        print(f"   {scenario}: {count}개")
    
    # 샘플 테스트
    print(f"\\n🧪 샘플 데이터 테스트:")
    sample = memory_efficient_dataset[0]
    for key, value in sample.items():
        if isinstance(value, torch.Tensor):
            print(f"   {key}: {value.shape} ({value.dtype})")
        else:
            print(f"   {key}: {type(value)}")
    
    clear_gpu_memory()
    print("\\n✅ 더미 데이터 기반 메모리 효율적인 설정 완료!")
    
else:
    print("❌ 더미 데이터셋도 비어있습니다. WindowChunkAdapter 로직을 확인해야 합니다.")


In [ ]:
# Cell 17: 🔄 실제 데이터로 메모리 효율적인 데이터셋 재생성

# WindowChunkAdapter 모듈 리로드
import importlib
import sys
if 'window_chunk_adapter' in sys.modules:
    importlib.reload(sys.modules['window_chunk_adapter'])

# WindowChunkAdapter 다시 로드
WindowChunkAdapter = load_class_from_file(
    'window_chunk_adapter',
    str(ROOT_DIR / 'data' / 'window_chunk_adapter.py'),
    'WindowChunkAdapter'
)

print("🔄 실제 데이터로 메모리 효율적인 데이터셋 재생성 중...")
print(f"📊 데이터 경로: {DATA_DIR}")

# 기존 데이터셋 정리
for var_name in ['memory_efficient_dataset', 'dataset']:
    if var_name in locals():
        del locals()[var_name]

clear_gpu_memory()

# 실제 데이터로 메모리 효율적인 데이터셋 생성
try:
    memory_efficient_dataset = WindowChunkAdapter(
        data_dir=str(DATA_DIR),
        window_size=8,   # 16 -> 8로 줄임 (메모리 절약)
        chunk_size=2,
        image_processor=kosmos_processor.image_processor,
        normalize_actions=True
    )
    
    print(f"\\n📊 실제 데이터 기반 메모리 효율적인 데이터셋 정보:")
    print(f"   총 샘플: {len(memory_efficient_dataset)}개")
    print(f"   Window Size: {memory_efficient_dataset.window_size} 프레임")
    print(f"   Chunk Size: {memory_efficient_dataset.chunk_size} 프레임")
    print(f"   Total Sequence: {memory_efficient_dataset.window_size + memory_efficient_dataset.chunk_size} 프레임")
    
    if len(memory_efficient_dataset) > 0:
        # 메모리 효율적인 데이터로더
        memory_efficient_dataloader = DataLoader(
            memory_efficient_dataset, 
            batch_size=1,  # 최소 배치 크기 (메모리 절약)
            shuffle=True, 
            num_workers=0  # 메모리 절약
        )
        
        print(f"   배치 크기: {memory_efficient_dataloader.batch_size}")
        print(f"   총 배치 수: {len(memory_efficient_dataloader)}")
        
        # 시나리오 분포 확인
        scenarios = [memory_efficient_dataset.samples[i]['scenario'] for i in range(min(len(memory_efficient_dataset), 100))]
        from collections import Counter
        scenario_counts = Counter(scenarios)
        
        print(f"\\n🎯 시나리오별 분포 (처음 100개):")
        for scenario, count in scenario_counts.items():
            print(f"   {scenario}: {count}개")
        
        # 샘플 테스트
        print(f"\\n🧪 샘플 데이터 테스트:")
        sample = memory_efficient_dataset[0]
        for key, value in sample.items():
            if isinstance(value, torch.Tensor):
                print(f"   {key}: {value.shape} ({value.dtype})")
            else:
                print(f"   {key}: {type(value)} = {value}")
        
        clear_gpu_memory()
        print("\\n✅ 실제 데이터 기반 메모리 효율적인 설정 완료!")
        
    else:
        print("❌ 실제 데이터셋에서 유효한 샘플을 찾을 수 없습니다.")
        print("   WindowChunkAdapter의 필터링 로직을 확인해야 합니다.")
        
        # 디버깅: 원본 H5 파일 몇 개 확인
        h5_files = list(DATA_DIR.glob("*.h5"))[:5]
        print(f"\\n🔍 디버깅: H5 파일 샘플 확인 (처음 5개):")
        
        import h5py
        for h5_file in h5_files:
            try:
                with h5py.File(h5_file, 'r') as f:
                    keys = list(f.keys())
                    attrs = dict(f.attrs)
                    print(f"   {h5_file.name}:")
                    print(f"     Keys: {keys}")
                    if 'images' in keys:
                        print(f"     Images: {f['images'].shape}")
                    if 'actions' in keys:
                        print(f"     Actions: {f['actions'].shape}")
                    print(f"     Attrs: {attrs}")
            except Exception as e:
                print(f"   {h5_file.name}: 오류 - {e}")

except Exception as e:
    print(f"❌ 데이터셋 생성 실패: {e}")
    print("\\n🔍 에러 상세:")
    import traceback
    traceback.print_exc()


In [ ]:
# Cell 18: 🚀 RoboVLMs 스타일 Mobile VLA 트레이닝

# RoboVLMs 스타일로 리팩토링된 모듈 임포트
print("🔄 RoboVLMs 스타일 Mobile VLA 모듈 로딩 중...")

# 경로 추가
import sys
sys.path.append(str(ROOT_DIR))

# 리팩토링된 모듈 임포트
try:
    from robovlms.data import MobileVLADataset
    from robovlms.train import MobileVLATrainer, ActionLossTracker
    print("✅ Mobile VLA 모듈 로딩 완료!")
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("   기존 RoboVLMs 환경에서 실행하거나 경로를 확인해주세요.")
    raise

# 기존 변수 정리
clear_gpu_memory()

# MobileVLADataset 초기화 (RoboVLMs 호환)
print("\\n📊 RoboVLMs 스타일 MobileVLADataset 초기화 중...")

try:
    # 토크나이저 설정 (RoboVLMs 방식)
    tokenizer_config = {
        "tokenizer_type": "kosmos",
        "max_text_len": 512,
        "tokenizer_id": "microsoft/kosmos-2-patch14-224"
    }
    
    # 데이터셋 설정
    dataset = MobileVLADataset(
        data_dir=str(DATA_DIR),
        model_name="kosmos",
        mode="train",
        organize_type="segment",  # RoboVLMs 호환
        window_size=8,  # 메모리 효율성
        fwd_pred_next_n=2,
        discrete=False,  # 연속 액션
        norm_action=True,
        image_history=True,
        action_history=True,
        tokenizer=tokenizer_config,
        rgb_pad=-1,  # 데이터 증강 비활성화
        gripper_pad=-1
    )
    
    print(f"✅ MobileVLADataset 초기화 완료!")
    print(f"   총 에피소드: {len(dataset)}개")
    
    if len(dataset) > 0:
        # 데이터로더 생성 (메모리 효율성)
        from torch.utils.data import DataLoader
        
        dataloader = DataLoader(
            dataset,
            batch_size=1,  # 메모리 절약
            shuffle=True,
            num_workers=0,  # 멀티프로세싱 비활성화 (안정성)
            collate_fn=dataset.collater  # RoboVLMs collater 사용
        )
        
        print(f"   배치 수: {len(dataloader)}개")
        
        # 샘플 데이터 확인
        print(f"\\n🧪 첫 번째 배치 데이터 구조 확인:")
        sample_batch = next(iter(dataloader))
        for key, value in sample_batch.items():
            if isinstance(value, torch.Tensor):
                print(f"   {key}: {value.shape} ({value.dtype})")
            elif isinstance(value, list):
                print(f"   {key}: list[{len(value)}] - {type(value[0]) if value else 'empty'}")
            else:
                print(f"   {key}: {type(value)}")
        
        print("\\n✅ RoboVLMs 스타일 데이터 로딩 완료!")
        
    else:
        print("❌ 데이터셋이 비어있습니다. 데이터 경로와 형식을 확인해주세요.")
        
except Exception as e:
    print(f"❌ 데이터셋 초기화 실패: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Cell 19: 🎯 RoboVLMs 스타일 Mobile VLA 트레이너 초기화 및 학습

if 'dataset' in locals() and len(dataset) > 0:
    print("🎯 MobileVLATrainer 초기화 중...")
    
    # 트레이너 설정
    trainer = MobileVLATrainer(
        model_name="microsoft/kosmos-2-patch14-224",
        action_dim=3,  # Mobile VLA: [linear_x, linear_y, angular_z]
        window_size=8,
        chunk_size=2,
        learning_rate=1e-4,
        device="cuda" if torch.cuda.is_available() else "cpu",
        precision="fp16"  # Mixed precision
    )
    
    print(f"✅ 트레이너 초기화 완료!")
    print(f"   디바이스: {trainer.device}")
    print(f"   모델 파라미터: {sum(p.numel() for p in trainer.model.parameters()):,}개")
    
    # 손실 추적기
    loss_tracker = ActionLossTracker()
    
    print("\\n🚀 RoboVLMs 스타일 학습 시작!")
    
    def train_mobile_vla_robovlms_style(
        trainer, 
        dataloader, 
        num_epochs=3, 
        log_interval=5,
        save_interval=50,
        checkpoint_dir="checkpoints"
    ):
        \"\"\"RoboVLMs 스타일 학습 함수\"\"\"
        
        # 체크포인트 디렉토리 생성
        checkpoint_path = ROOT_DIR / checkpoint_dir
        checkpoint_path.mkdir(exist_ok=True)
        
        print(f"🏃‍♂️ 학습 시작: {num_epochs} 에포크, {len(dataloader)} 배치")
        print(f"📁 체크포인트 저장 경로: {checkpoint_path}")
        
        all_losses = []
        best_loss = float('inf')
        
        try:
            for epoch in range(num_epochs):
                print(f"\\n📅 Epoch {epoch+1}/{num_epochs}")
                epoch_losses = []
                
                for step, batch in enumerate(dataloader):
                    try:
                        # 학습 스텝
                        loss_dict = trainer.train_step(batch)
                        loss_tracker.update(loss_dict)
                        epoch_losses.append(loss_dict["total_loss"])
                        
                        # 로깅
                        if (step + 1) % log_interval == 0:
                            avg_metrics = loss_tracker.get_averages(last_n=log_interval)
                            print(f"  Step {step+1:3d}: "
                                  f"Loss={avg_metrics['avg_total_loss']:.4f}, "
                                  f"MAE={avg_metrics['avg_mae']:.4f}, "
                                  f"LR={avg_metrics['current_lr']:.2e}")
                        
                        # 체크포인트 저장
                        if (step + 1) % save_interval == 0:
                            current_loss = loss_tracker.get_averages(last_n=10)['avg_total_loss']
                            if current_loss < best_loss:
                                best_loss = current_loss
                                checkpoint_file = checkpoint_path / f"mobile_vla_best_epoch{epoch+1}_step{step+1}.pt"
                                trainer.save_checkpoint(
                                    str(checkpoint_file), 
                                    epoch=epoch+1,
                                    metrics={"best_loss": best_loss, "step": step+1}
                                )
                                print(f"💾 Best 모델 저장: {checkpoint_file.name} (Loss: {best_loss:.4f})")
                        
                        all_losses.append(loss_dict)
                        
                    except Exception as e:
                        print(f"⚠️ Step {step+1} 실패: {e}")
                        clear_gpu_memory()  # OOM 복구 시도
                        continue
                
                # 에포크 종료 요약
                if epoch_losses:
                    epoch_avg_loss = sum(epoch_losses) / len(epoch_losses)
                    print(f"📊 Epoch {epoch+1} 완료: 평균 Loss = {epoch_avg_loss:.4f}")
                    
                    # 에포크별 체크포인트
                    epoch_checkpoint = checkpoint_path / f"mobile_vla_epoch{epoch+1}.pt"
                    trainer.save_checkpoint(
                        str(epoch_checkpoint),
                        epoch=epoch+1,
                        metrics={"epoch_avg_loss": epoch_avg_loss}
                    )
                else:
                    print(f"⚠️ Epoch {epoch+1}: 유효한 배치가 없었습니다.")
                
                clear_gpu_memory()
        
        except KeyboardInterrupt:
            print("\\n⏹️ 학습이 사용자에 의해 중단되었습니다.")
        except Exception as e:
            print(f"\\n❌ 학습 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
        
        return all_losses, loss_tracker
    
    # 실제 학습 실행
    print("\\n" + "="*50)
    print("🎓 Mobile VLA RoboVLMs 스타일 학습 실행")
    print("="*50)
    
    losses, tracker = train_mobile_vla_robovlms_style(
        trainer=trainer,
        dataloader=dataloader,
        num_epochs=2,  # 빠른 테스트
        log_interval=3,
        save_interval=20,
        checkpoint_dir="mobile_vla_checkpoints"
    )
    
    print("\\n🎉 학습 완료!")
    
    # 최종 통계
    if tracker.losses:
        final_metrics = tracker.get_averages()
        print(f"\\n📈 최종 학습 통계:")
        print(f"   평균 Loss: {final_metrics['avg_total_loss']:.4f}")
        print(f"   평균 MAE: {final_metrics['avg_mae']:.4f}")
        print(f"   총 스텝: {final_metrics['steps']}")
        print(f"   최종 학습률: {final_metrics['current_lr']:.2e}")
        
else:
    print("❌ 유효한 데이터셋이 없어 학습을 실행할 수 없습니다.")
    print("   이전 셀에서 데이터셋을 먼저 초기화해주세요.")


In [18]:
# Cell 20: 🧪 PyArrow 호환 Mobile VLA 완전 리셋 및 테스트

print("🔄 완전 리셋 및 환경 재구성...")

# 메모리 정리
clear_gpu_memory()

# 모든 관련 모듈 언로드
import sys
modules_to_reload = [m for m in sys.modules.keys() if 'robovlms' in m or 'mobile_vla' in m]
for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

print("✅ 모듈 캐시 정리 완료")

# 의존성 버전 최종 확인
try:
    import pyarrow
    import datasets
    print(f"✅ PyArrow: {pyarrow.__version__}")
    print(f"✅ Datasets: {datasets.__version__}")
    
    # 호환성 테스트
    test_dataset = datasets.Dataset.from_dict({"test": [1, 2, 3]})
    print("✅ Datasets/PyArrow 호환성 확인")
    
except Exception as e:
    print(f"❌ 의존성 문제: {e}")
    raise

# 경로 재설정
sys.path.insert(0, str(ROOT_DIR))

# 리팩토링된 모듈 재임포트
print("\\n🔄 Mobile VLA 모듈 재로딩...")
try:
    from robovlms.data import MobileVLADataset
    from robovlms.train import MobileVLATrainer, ActionLossTracker
    print("✅ 모든 모듈 성공적으로 로드!")
except ImportError as e:
    print(f"❌ 임포트 실패: {e}")
    import traceback
    traceback.print_exc()
    raise

# 실제 데이터셋 테스트
print(f"\\n📊 실제 데이터셋 테스트 시작...")
print(f"   데이터 경로: {DATA_DIR}")

try:
    # 간단한 설정으로 데이터셋 생성
    dataset = MobileVLADataset(
        data_dir=str(DATA_DIR),
        model_name="kosmos",
        mode="train",
        window_size=8,
        fwd_pred_next_n=2,
        discrete=False,
        tokenizer=None  # 간단히 None으로 설정
    )
    
    print(f"✅ 데이터셋 생성 성공!")
    print(f"   총 에피소드: {len(dataset)}개")
    
    if len(dataset) > 0:
        # 첫 번째 샘플 테스트
        sample = dataset[0]
        print(f"\\n📋 첫 번째 샘플:")
        print(f"   시나리오: {sample['scenario']}")
        print(f"   태스크: {sample['task_description']}")
        print(f"   이미지 수: {len(sample['images'])}")
        print(f"   액션 shape: {sample['actions'].shape}")
        
        # 커스텀 collate function으로 DataLoader 생성 (PIL 이미지 처리)
        def simple_collate_fn(batch):
            """간단한 PIL 이미지 처리 collate function"""
            import torch
            from torchvision import transforms
            
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
            collated = {}
            for key in batch[0].keys():
                if key == 'images':
                    # PIL 이미지를 텐서로 변환
                    all_tensors = []
                    for item in batch:
                        seq_tensors = []
                        for pil_img in item[key]:
                            tensor_img = transform(pil_img)
                            seq_tensors.append(tensor_img)
                        all_tensors.append(torch.stack(seq_tensors))
                    collated[key] = torch.stack(all_tensors)
                elif key == 'actions':
                    collated[key] = torch.stack([torch.tensor(item[key], dtype=torch.float32) for item in batch])
                elif key == 'episode_mask':
                    collated[key] = torch.stack([torch.tensor(item[key], dtype=torch.bool) for item in batch])
                else:
                    collated[key] = [item[key] for item in batch]
            return collated
        
        from torch.utils.data import DataLoader
        dataloader = DataLoader(
            dataset, 
            batch_size=1, 
            shuffle=False, 
            num_workers=0,
            collate_fn=simple_collate_fn  # 커스텀 collate function 사용
        )
        
        # 첫 번째 배치 테스트
        print("   배치 생성 중 (PIL → Tensor 변환)...")
        first_batch = next(iter(dataloader))
        print(f"\\n🎯 배치 테스트:")
        for key, value in first_batch.items():
            if hasattr(value, 'shape'):
                print(f"   {key}: {value.shape}")
            elif isinstance(value, (list, tuple)):
                print(f"   {key}: {type(value).__name__}[{len(value)}]")
            else:
                print(f"   {key}: {type(value).__name__}")
        
        print("\\n🎉 모든 테스트 통과! 학습 준비 완료!")
        
    else:
        print("❌ 에피소드가 없습니다. 데이터 경로를 확인해주세요.")
        
except Exception as e:
    print(f"❌ 데이터셋 테스트 실패: {e}")
    import traceback
    traceback.print_exc()
    dataset = None
    dataloader = None


🔄 완전 리셋 및 환경 재구성...
🧹 GPU 메모리 정리 중...
   할당된 메모리: 6.24 GB
   캐시된 메모리: 6.26 GB
✅ 모듈 캐시 정리 완료
✅ PyArrow: 14.0.2
✅ Datasets: 2.12.0
✅ Datasets/PyArrow 호환성 확인
\n🔄 Mobile VLA 모듈 재로딩...
✅ 모든 모듈 성공적으로 로드!
\n📊 실제 데이터셋 테스트 시작...
   데이터 경로: /home/billy/25-1kp/vla/ROS_action/mobile_vla_dataset
✅ 데이터셋 생성 성공!
   총 에피소드: 72개
\n📋 첫 번째 샘플:
   시나리오: 2box_left_vertical
   태스크: Navigate around the two box obstacles by going left to track the target cup
   이미지 수: 18
   액션 shape: (18, 3)
❌ 데이터셋 테스트 실패: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>


Traceback (most recent call last):
  File "/home/billy/.cache/pypoetry/virtualenvs/mobile-vla-FNblWQUj-py3.10/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 182, in collate
    clone[i] = collate(samples, collate_fn_map=collate_fn_map)
  File "/home/billy/.cache/pypoetry/virtualenvs/mobile-vla-FNblWQUj-py3.10/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 191, in collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/billy/.cache/pypoetry/virtualenvs/mobile-vla-FNblWQUj-py3.10/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 154, in collate
    clone.update({key: collate([d[key] for d in batch], collate_fn_map=collate_fn_map) for key in elem})
  File "/

In [ ]:
# Cell 21: 🚀 최종 Mobile VLA 학습 실행 (PyArrow 호환)

if 'dataset' in locals() and dataset is not None and len(dataset) > 0:
    print("🎯 MobileVLATrainer 초기화 및 학습 시작...")
    
    try:
        # 트레이너 초기화
        trainer = MobileVLATrainer(
            model_name="microsoft/kosmos-2-patch14-224",
            action_dim=3,  # Mobile VLA: [linear_x, linear_y, angular_z]
            window_size=8,
            chunk_size=2,
            learning_rate=1e-4,
            device="cuda" if torch.cuda.is_available() else "cpu",
            precision="fp16"  # Mixed precision
        )
        
        print(f"✅ 트레이너 초기화 완료!")
        print(f"   디바이스: {trainer.device}")
        print(f"   모델 파라미터: {sum(p.numel() for p in trainer.model.parameters()):,}개")
        
        # 손실 추적기
        loss_tracker = ActionLossTracker()
        
        # 학습 실행
        print("\\n🚀 Mobile VLA 학습 시작!")
        
        num_epochs = 3  # 테스트용으로 짧게
        save_interval = 1
        log_interval = 5
        
                 # 학습용 DataLoader 재생성 (collate_fn 포함)
         print("   학습용 DataLoader 생성 중...")
         from torch.utils.data import DataLoader
         train_dataloader = DataLoader(
             dataset,
             batch_size=1,
             shuffle=True,
             num_workers=0,
             collate_fn=dataset.collater,  # 중요: collate_fn 사용
             pin_memory=False,
             drop_last=True
         )
         print(f"   DataLoader 생성 완료 (총 {len(train_dataloader)}개 배치)")
         
         for epoch in range(num_epochs):
             print(f"\\n📊 Epoch {epoch+1}/{num_epochs}")
             epoch_losses = []
             
             for step, batch in enumerate(train_dataloader):
                try:
                    # 학습 스텝 실행
                    loss_dict = trainer.train_step(batch)
                    
                    # 손실 추적
                    loss_tracker.update(loss_dict)
                    epoch_losses.append(loss_dict['total_loss'])
                    
                    # 주기적 로깅
                    if (step + 1) % log_interval == 0:
                        current_metrics = loss_tracker.get_averages()
                        print(f"   Step {step+1:3d}: "
                              f"Loss={loss_dict['total_loss']:.4f}, "
                              f"MAE={loss_dict.get('mae_avg', 0):.4f}, "
                              f"LR={current_metrics['current_lr']:.2e}")
                    
                    # 메모리 관리 (주기적으로 정리)
                    if (step + 1) % 10 == 0:
                        clear_gpu_memory()
                        
                except Exception as e:
                    print(f"❌ 스텝 {step+1} 실패: {e}")
                    if "out of memory" in str(e).lower():
                        print("   GPU 메모리 부족! 배치 크기를 줄이거나 window_size를 줄여주세요.")
                        clear_gpu_memory()
                        break
                    else:
                        continue
            
            # 에포크 요약
            if epoch_losses:
                avg_epoch_loss = sum(epoch_losses) / len(epoch_losses)
                print(f"   📈 Epoch {epoch+1} 평균 Loss: {avg_epoch_loss:.4f}")
                
                # 체크포인트 저장
                if (epoch + 1) % save_interval == 0:
                    checkpoint_path = f"mobile_vla_epoch_{epoch+1}.pt"
                    trainer.save_checkpoint(checkpoint_path)
                    print(f"   💾 체크포인트 저장: {checkpoint_path}")
        
        print("\\n🎉 학습 완료!")
        
        # 최종 통계
        if loss_tracker.losses:
            final_metrics = loss_tracker.get_averages()
            print(f"\\n📈 최종 학습 통계:")
            print(f"   평균 Total Loss: {final_metrics['avg_total_loss']:.4f}")
            print(f"   평균 MAE: {final_metrics['avg_mae']:.4f}")
            print(f"   총 스텝: {final_metrics['steps']}")
            print(f"   최종 학습률: {final_metrics['current_lr']:.2e}")
            
            # 간단한 성능 분석
            print(f"\\n🔍 성능 분석:")
            print(f"   Loss 추이: {loss_tracker.losses[-10:] if len(loss_tracker.losses) >= 10 else loss_tracker.losses}")
            
        # 최종 메모리 정리
        clear_gpu_memory()
        
    except Exception as e:
        print(f"❌ 학습 실행 실패: {e}")
        import traceback
        traceback.print_exc()
        clear_gpu_memory()
        
else:
    print("❌ 유효한 데이터셋이 없어 학습을 실행할 수 없습니다.")
    print("   이전 셀(Cell 20)에서 데이터셋을 먼저 초기화해주세요.")
    print(f"   현재 dataset 상태: {'정의됨' if 'dataset' in locals() else '정의되지 않음'}")
    if 'dataset' in locals():
        print(f"   데이터셋 길이: {len(dataset) if dataset is not None else 'None'}")


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 81)

: 

In [20]:
# Cell 22: 🎯 Collate Function 문제 해결 및 최종 데이터셋 준비

print("🔧 Collate Function 문제 해결...")

# 커스텀 collate 함수 정의 (PIL 이미지 처리 완벽 지원)
def mobile_vla_collate_fixed(batch):
    """PIL 이미지를 텐서로 변환하는 커스텀 collate 함수 (완전 수정판)"""
    import torch
    from torchvision import transforms
    
    # 이미지 전처리 파이프라인 (Kosmos 호환)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    collated = {}
    
    for key in batch[0].keys():
        if key == 'images':
            # PIL 이미지들을 텐서로 변환
            image_tensors = []
            for item in batch:
                seq_tensors = []
                for pil_img in item[key]:
                    tensor_img = transform(pil_img)
                    seq_tensors.append(tensor_img)
                # [T, C, H, W] 스택
                if seq_tensors:
                    stacked = torch.stack(seq_tensors)
                    image_tensors.append(stacked)
                else:
                    # 빈 시퀀스 처리
                    image_tensors.append(torch.empty(0, 3, 224, 224))
            # [B, T, C, H, W] 배치
            if image_tensors and image_tensors[0].numel() > 0:
                collated[key] = torch.stack(image_tensors)
            else:
                collated[key] = torch.empty(len(batch), 0, 3, 224, 224)
                
        elif key == 'actions':
            # 액션 텐서화
            action_tensors = []
            for item in batch:
                if isinstance(item[key], torch.Tensor):
                    action_tensors.append(item[key])
                else:
                    action_tensors.append(torch.tensor(item[key], dtype=torch.float32))
            collated[key] = torch.stack(action_tensors)
            
        elif key == 'episode_mask':
            # 마스크 텐서화  
            mask_tensors = []
            for item in batch:
                if isinstance(item[key], torch.Tensor):
                    mask_tensors.append(item[key])
                else:
                    mask_tensors.append(torch.tensor(item[key], dtype=torch.bool))
            collated[key] = torch.stack(mask_tensors)
            
        else:
            # 문자열 등은 리스트로 유지
            collated[key] = [item[key] for item in batch]
    
    return collated

# 데이터셋이 이미 로드되어 있는지 확인
if 'dataset' in locals() and dataset is not None and len(dataset) > 0:
    print(f"✅ 기존 데이터셋 사용: {len(dataset)}개 에피소드")
    
    # 수정된 collate 함수로 DataLoader 재생성
    from torch.utils.data import DataLoader
    
    dataloader = DataLoader(
        dataset, 
        batch_size=1, 
        shuffle=False, 
        num_workers=0,
        collate_fn=mobile_vla_collate_fixed  # 수정된 커스텀 collate 함수
    )
    
    print("🔄 배치 테스트 실행 중...")
    
    try:
        # 첫 번째 배치 테스트
        first_batch = next(iter(dataloader))
        
        print("✅ 배치 로드 성공!")
        print("\\n🎯 최종 배치 구조:")
        for key, value in first_batch.items():
            if hasattr(value, 'shape'):
                print(f"   {key}: {value.shape} ({value.dtype})")
            elif isinstance(value, (list, tuple)):
                print(f"   {key}: {type(value).__name__}[{len(value)}]")
                if len(value) > 0:
                    print(f"     └─ 첫 번째 항목: {value[0]}")
            else:
                print(f"   {key}: {type(value).__name__}")
        
        print("\\n🎉 모든 Collate 테스트 통과! 학습 준비 완료!")
        
        # 간단한 Window/Chunk 확인
        images_shape = first_batch['images'].shape  # [B, T, C, H, W]
        actions_shape = first_batch['actions'].shape  # [B, T, action_dim]
        
        window_size = 8
        chunk_size = 2
        
        print(f"\\n📊 Window/Chunk 분석:")
        print(f"   이미지 시퀀스: {images_shape} (배치={images_shape[0]}, 시간={images_shape[1]})")
        print(f"   액션 시퀀스: {actions_shape} (배치={actions_shape[0]}, 시간={actions_shape[1]})")
        print(f"   Window Size: {window_size} (과거 관찰)")
        print(f"   Chunk Size: {chunk_size} (미래 예측)")
        
        if images_shape[1] >= window_size + chunk_size:
            print(f"   ✅ 충분한 시퀀스 길이: {images_shape[1]} >= {window_size + chunk_size}")
        else:
            print(f"   ⚠️ 시퀀스 길이 부족: {images_shape[1]} < {window_size + chunk_size}")
            
    except Exception as e:
        print(f"❌ 배치 테스트 실패: {e}")
        import traceback
        traceback.print_exc()
        dataloader = None
        
else:
    print("❌ 데이터셋이 없습니다. 이전 셀을 먼저 실행해주세요.")
    dataloader = None


🔧 Collate Function 문제 해결...
❌ 데이터셋이 없습니다. 이전 셀을 먼저 실행해주세요.


In [21]:
# Cell 22: 🔧 PIL Image 처리를 위한 커스텀 DataLoader

print("🔧 PIL Image 문제 해결을 위한 커스텀 DataLoader 구성...")

if 'dataset' in locals() and dataset is not None and len(dataset) > 0:
    
    def mobile_vla_collate_fn(batch):
        """Mobile VLA용 커스텀 collate function - PIL Image 안전 처리"""
        import torch
        from PIL import Image
        import numpy as np
        
        print(f"🔍 배치 크기: {len(batch)}")
        
        # 첫 번째 샘플 구조 확인
        if batch:
            sample = batch[0]
            print(f"📋 샘플 키: {list(sample.keys())}")
            for key, value in sample.items():
                if key == 'images':
                    print(f"   {key}: {len(value)}개 이미지 (타입: {type(value[0]) if value else 'empty'})")
                elif hasattr(value, 'shape'):
                    print(f"   {key}: {value.shape}")
                else:
                    print(f"   {key}: {type(value).__name__}")
        
        collated = {}
        
        for key in batch[0].keys():
            values = [item[key] for item in batch]
            
            if key == 'images':
                # PIL 이미지를 처리
                all_image_tensors = []
                for img_list in values:  # 각 배치의 이미지 리스트
                    batch_images = []
                    for img in img_list:
                        if isinstance(img, Image.Image):
                            # PIL -> numpy -> tensor (정규화 포함)
                            img_array = np.array(img)
                            if img_array.ndim == 3:  # RGB
                                img_array = img_array.transpose(2, 0, 1)  # HWC -> CHW
                            img_tensor = torch.from_numpy(img_array).float() / 255.0
                            batch_images.append(img_tensor)
                        elif isinstance(img, torch.Tensor):
                            batch_images.append(img)
                        else:
                            print(f"⚠️ 지원하지 않는 이미지 타입: {type(img)}")
                            continue
                    
                    if batch_images:
                        all_image_tensors.append(torch.stack(batch_images))
                
                # 최종 배치 텐서 생성
                if all_image_tensors:
                    collated[key] = torch.stack(all_image_tensors)  # [B, T, C, H, W]
                    print(f"   ✅ {key} 처리 완료: {collated[key].shape}")
                else:
                    collated[key] = torch.empty(0)
                    print(f"   ⚠️ {key} 비어있음")
                    
            elif key == 'actions':
                # 액션 텐서 처리
                if isinstance(values[0], torch.Tensor):
                    collated[key] = torch.stack(values)
                else:
                    collated[key] = torch.stack([torch.tensor(v, dtype=torch.float32) for v in values])
                print(f"   ✅ {key} 처리 완료: {collated[key].shape}")
                
            elif key == 'episode_mask':
                # 마스크 처리
                if isinstance(values[0], torch.Tensor):
                    collated[key] = torch.stack(values)
                else:
                    collated[key] = torch.stack([torch.tensor(v, dtype=torch.bool) for v in values])
                print(f"   ✅ {key} 처리 완료: {collated[key].shape}")
                
            elif isinstance(values[0], torch.Tensor):
                # 기타 텐서들
                collated[key] = torch.stack(values)
                print(f"   ✅ {key} 처리 완료: {collated[key].shape}")
            else:
                # 문자열이나 기타 객체들
                collated[key] = values
                print(f"   ✅ {key} 처리 완료: {type(values).__name__}[{len(values)}]")
        
        return collated
    
    # 커스텀 collate function을 사용하는 DataLoader 생성
    try:
        safe_dataloader = DataLoader(
            dataset,
            batch_size=1,  # 메모리 효율성
            shuffle=False,
            num_workers=0,  # 멀티프로세싱 비활성화
            pin_memory=False,
            drop_last=True,
            collate_fn=mobile_vla_collate_fn  # 커스텀 collate function
        )
        
        print(f"\\n✅ 안전한 DataLoader 생성 완료!")
        print(f"   배치 크기: 1")
        print(f"   총 배치 수: {len(safe_dataloader)}개")
        
        # 실제 배치 테스트
        print("\\n🧪 실제 배치 로딩 테스트...")
        first_batch = next(iter(safe_dataloader))
        
        print(f"\\n🎯 최종 배치 구조:")
        for key, value in first_batch.items():
            if hasattr(value, 'shape'):
                print(f"   {key}: {value.shape} ({value.dtype})")
            elif isinstance(value, (list, tuple)):
                print(f"   {key}: {type(value).__name__}[{len(value)}]")
            else:
                print(f"   {key}: {type(value).__name__}")
        
        print("\\n🎉 DataLoader 테스트 성공! 학습 준비 완료!")
        
        # 전역 변수에 할당
        dataloader = safe_dataloader
        
    except Exception as e:
        print(f"❌ 안전한 DataLoader 생성 실패: {e}")
        import traceback
        traceback.print_exc()
        dataloader = None
        
else:
    print("❌ 유효한 데이터셋이 없습니다. 이전 셀을 먼저 실행해주세요.")


🔧 PIL Image 문제 해결을 위한 커스텀 DataLoader 구성...
❌ 유효한 데이터셋이 없습니다. 이전 셀을 먼저 실행해주세요.
